## Importing necessary Datasets

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [3]:
from tensorflow.keras.applications import InceptionV3

In [4]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Setting the parameters

In [5]:
batch_size = 32
epochs = 10
image_size = (224, 224)

## Loading Data from Dataset

In [6]:
train_data_dir = 'food_dataset\\train'
validation_data_dir = 'food_dataset\\validation'
test_data_dir = 'food_dataset\\test'

### Data augmentation and preprocessing

In [7]:
train_datagen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

Found 3115 images belonging to 36 classes.


In [9]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

Found 351 images belonging to 36 classes.


These Data Generators are useful when dealing with loading large batches of images like here 

## Loading Pre-trained Inception V3 model 

Load the InceptionV3 model

In [10]:
base_model = InceptionV3(weights='imagenet', include_top=False)

87910968/87910968 [==============================] - 9s 0us/step


Add custom layers for classification task

In [11]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

Combine the base model with custom layers

In [12]:
model = Model(inputs=base_model.input, outputs=predictions)

Compile the model

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training

Train the model

In [14]:
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/10
10/98 [==>...........................] - ETA: 8:03 - loss: 3.5218 - accuracy: 0.1656

C:\Users\kommi\anaconda3\envs\Tf-new\lib\site-packages\PIL\Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


98/98 [==============================] - 698s 7s/step - loss: 2.3010 - accuracy: 0.3759 - val_loss: 7.2991 - val_accuracy: 0.1311
Epoch 2/10
98/98 [==============================] - 548s 6s/step - loss: 1.5185 - accuracy: 0.5596 - val_loss: 8.6640 - val_accuracy: 0.1795
Epoch 3/10
98/98 [==============================] - 562s 6s/step - loss: 1.2202 - accuracy: 0.6302 - val_loss: 4.1449 - val_accuracy: 0.3590
Epoch 4/10
98/98 [==============================] - 500s 5s/step - loss: 1.0631 - accuracy: 0.6844 - val_loss: 2.2638 - val_accuracy: 0.4501
Epoch 5/10
98/98 [==============================] - 514s 5s/step - loss: 0.9551 - accuracy: 0.7130 - val_loss: 1.2652 - val_accuracy: 0.6838
Epoch 6/10
98/98 [==============================] - 494s 5s/step - loss: 0.8114 - accuracy: 0.7457 - val_loss: 1.1222 - val_accuracy: 0.6838
Epoch 7/10
98/98 [==============================] - 474s 5s/step - loss: 0.8108 - accuracy: 0.7432 - val_loss: 1.8140 - val_accuracy: 0.5014
Epoch 8/10
98/98 [======

### Saving Model

In [15]:
model.save('inceptionv3_food.h5')

In [16]:
model.save_weights('inceptionv3_food_weights.h5')

### Testing Model

Evaluate the model on the test data

In [17]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

Found 359 images belonging to 36 classes.


In [18]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

12/12 [==============================] - 57s 5s/step - loss: 1.3824 - accuracy: 0.6741
Test loss: 1.3823548555374146, Test accuracy: 0.6740947365760803


### Observation:
### As seen here accuracy isn't great so we will fine tune it and train more to increase accuracy
